# RSTT Tutorial 4 - Implementations Verifications

## 1. StagedEvent 

Competition classes are centrals to the package. They depends on ranking for seedings of participants. They generate games with a Solver and interpret the outcomes. All abstractions come together and work together during the run() execution which can make it hard to debug. Best is to first test it in conjonction with standard RSTT components such as BasicPlayer, BTRanking and BetterWin.

### 1.3 Standard usage 

**Ensure that:**
- 1) Your class can be instanciated with at least a name, seedings and a solver
- 2) Player can be registered
- 3) The run() method can be called with no crash

In [1]:
# import your class
from project import StagedEvent

# rstt imports
from rstt import BasicPlayer, BTRanking, BetterWin
from rstt import RoundRobin, SingleEliminationBracket, DoubleEliminationBracket

# simulation parameters
teams = BasicPlayer.create(nb=8)
gt = BTRanking(name='GroundTruth', players=teams)
solver = BetterWin()

# stagedEvent parameters
tournaments = [RoundRobin, SingleEliminationBracket, DoubleEliminationBracket]
names = ['PlayIns', 'MainStage', 'PlayOffs']

# Integration test
test = StagedEvent(name='test', seeding=gt, solver=solver, tournaments=tournaments, stage_names=names)
print('Instanciation done!')

test.registration(teams)
print('Registration done!')

test.run()
print('It runs smoothly aswell, good job!')

Instanciation done!
Registration done!
It runs smoothly aswell, good job!


### 1.2. Competition Quality

In simulation, the role of a competition class is to output a small match dataset with a specific logic/structure and identify a winner among a predefine number of competitors. In our project, the resulting games are used to update the ranking, and the final placement is used to qualify teams to other events. These are two important features of the StagedEvent class. In the next cell we perform some 'integrity check' to ensure the implementation does what it is supposed to do.

**Ensure that:**
- 1) Stages are properly executed: each started(), were played entirely and the trophies() method was properly called.
- 2) The competition returns the right amout of games
- 3) The competition correctly build the final standing


**TIP:**
A simulation using the BetterWin solver and BasicPlayer with constant level is deterministic and always produces the same output.
Feel free to change the test/requierements, as long as you clearly define what your implementation should do!

In [2]:
# 1. check stages proper run
for stage in test.stages:
    assert stage.started()
    assert not stage.live()
    assert stage.over()

# 2. check return dataset size
'''target_games = []
for i, tournament in enumerate(tournaments):
    stage = tournament(name=f'test_stage_games_{i}', seeding=gt, solver=solver)
    stage.registration(teams)
    stage.run()
    target_games += stage.games()
assert len(test.games()) == len(target_games)'''

# 3. check that the stagedEvent standing matches the 'final stage' standing .
final = tournaments[-1](name='test_final_stage_standing', seeding=gt, solver=solver)
final.registration(teams)
final.run()
assert test.standing() == final.standing()

## 2. LeagueSystem

One restriction of the package is the type of Ranking *keys*, which must implement the SPlayer protocol. Since the Global power ranking rates also *Regions* and not just Teams, they must be represented by a Player class. The scene module contains a Region dataclass with value for region. The LeagueSystem class maps Region values with Player instances and corresponding Teams. The following cells consist in minimal unit test. 

**Ensure** That the next cells runs with no assertion errors.

In [3]:
from project import LeagueSystem
from project.scene import Region

# A LeagueSystem
r_size = 8
teams_by_leagues = {region: BasicPlayer.create(nb=r_size) for region in Region}
ecosystem = LeagueSystem(teams_by_leagues)

# 1. correct numbers of leagues
assert len(set(ecosystem.leagues())) == len(Region)

# 2. correct number of teams in each region
for region in Region:
    assert len(set(ecosystem.teams(region))) == r_size

# 3. correct total amount of teams
assert len(set(ecosystem.teams()) )== r_size*len(Region)

# 4. correct region of each team
mapping = {league: 0 for league in ecosystem.leagues()}
for region, teams in teams_by_leagues.items():
    for team in teams:
        assert ecosystem.region_of(team) == region
        mapping[ecosystem.league_of(ecosystem.region_of(team))] += 1

# 5. correct number of teams associated to a league
for league in ecosystem.leagues():
    assert mapping[league] == 8

## 3. Game History

The Global Power Ranking uses different window size of games for Region or Team ratings. 

**Ensure** That the next cells runs with no assertion errors.

In [4]:
from project.gpr import GameHistory
from project.gpr.utils import Modes

# instanciation
RegionRange, TeamRange = 2, 3
history = GameHistory(mode_range={Modes.League: RegionRange, Modes.Team: TeamRange})

# bunch of dummy events
cups = []
for i in range(max(RegionRange, TeamRange)*2):
    cup = RoundRobin(f'history {i}', gt, BetterWin())
    cup.registration(gt.players())
    cup.run()
    cups.append(cup)

    # check ordering of events
    history.add(cup)
    assert cup is history.window(Modes.League)[-1]
    assert cup is history.window(Modes.Team)[-1]

# check number of event returned
assert len(history.window(Modes.League)) == RegionRange
assert len(history.window(Modes.Team)) == TeamRange

# check accessibility of event
for cup in cups:
    assert cup is history.get_event(cup.name())

## 4. Power Score Formula

The formula is central to the GPR design. Thus, for evaluation purposes, we want it to be as much paramtreziable as possible. 

**Ensure** that the x and y values and the default ratings value are tunable in your RatingSystem Implementation

In [5]:
from project.gpr import RegionalRatings

from rstt.ranking import KeyModel


x = 0.8
y = 0.2
DefaultTeamRating, DefaultRegionRating = 1500, 1000
datamodel = RegionalRatings(ecosystem,
                            ratings ={
                                Modes.Team: KeyModel(default=DefaultTeamRating),
                                Modes.League: KeyModel(default=DefaultRegionRating)
                            },
                            x = x, y = y)

# test the GPR scoring formula
for team in ecosystem.teams():
    rating = datamodel.get(team)
    assert datamodel.ordinal(rating) == x * DefaultTeamRating + y * DefaultRegionRating

### MetaData

In [6]:
from rstt import GaussianPlayer

from project.model import MetaData
from project.scene import Role

# Instanciation
ROLES = {r: w for r, w in zip(Role, [0.1, 0.25, 0.2, 0.3, 0.15])}
BLevel, RLevel = 1500, 1500
Bs, Rs = 75, 75
BLUE = GaussianPlayer('BlueSide', mu=BLevel, sigma=Bs)
RED = GaussianPlayer('RedSide', mu=RLevel, sigma=Rs)
meta = MetaData(ROLES, BLUE, RED)

# Main feature usage
meta.update()

# check meta changes - side importance
assert BLevel != meta.blue().level()
assert RLevel != meta.red().level()

# check meta changes - role importance
for role in Role:
    assert meta.weights()[role] != ROLES[role]

In [7]:
meta.weights()

{<Role.Toplaner: 'Toplaner'>: 1.1259700621562017,
 <Role.Jungle: 'Jungler'>: 0.5656538990475783,
 <Role.Midlaner: 'Midlaner'>: 1.2481408926229227,
 <Role.Botlaner: 'Botlaner'>: 0.7498951627087104,
 <Role.Support: 'Support'>: 1.3103399834645864}

### LoLTeam && LoLSolver

In [8]:
from project.model import LoLTeam
from project.scene import Role

names_A = [f"{str(role)}_A" for role in Role]
names_B = [f"{str(role)}_B" for role in Role]
levels_A = [1000, 1200, 1400, 1600, 1800]
levels_B = [2000, 1700, 1300, 1100, 900]
TeamA = LoLTeam(name="TeamA", )

In [9]:
from rstt import BasicPlayer

player = BasicPlayer('pismice', 1200)

In [10]:
player.level()

1200

In [11]:
from rstt import BTRanking

pop = BasicPlayer.create(nb=10, level_params={'mu':2000, 'sigma':500})

In [12]:
gt = BTRanking('gt', pop)

In [13]:
gt[0].level()

2794.065826697194

In [14]:
pop[9].level()

2088.570630735534